<h2>Installing required packages</h2>

In [1]:
!pip install tensorflow

In [2]:
!pip install keras

In [3]:
!pip install matplotlib

In [4]:
!pip install pennylane --upgrade

<h2>Necessary imports</h2>

In [5]:
from keras.models import Sequential
from keras.layers import Dense

In [6]:
from sklearn import metrics


In [7]:
import json
from Filters import *
from Data_load import data_load
import matplotlib.pyplot as plt

import keras

from utils import *

import time

## Import the images
data = data_load()
train_images, train_labels, test_images, test_labels = data.data_mnist()

6000


In [8]:
print("shape of train images", train_images[0].shape)
print("shape of test images", test_images[0].shape)

shape of train images (28, 28, 1)
shape of test images (28, 28, 1)


In [9]:
print("length of train images", len(train_images))
print("length of test images", len(test_images))

length of train images 6000
length of test images 1000


<h2>Using 28x28 images</h2>

In [10]:
# This function returns the modified dataset
# It is a script to get a dataset of 2 labels only
def trainTestData(train_images, train_labels, test_images, test_labels, label1, label2, n_train, n_test):
    n_train_images, n_train_labels, n_test_images, n_test_labels = [], [], [], []
    i = 0
    while (n_train > 0 and i < len(train_images)):
        if train_labels[i] == label1:
            n_train_images.append(train_images[i])
            n_train_labels.append(label1)
            n_train -= 1
        if train_labels[i] == label2:
            n_train_images.append(train_images[i])
            n_train_labels.append(label2)
            n_train -= 1
        i = i+1
    i = 0
    while(n_test>0 and i < len(test_images)):
        if test_labels[i] == label1:
            n_test_images.append(test_images[i])
            n_test_labels.append(label1)
            n_test -= 1
        if test_labels[i] == label2:
            n_test_images.append(test_images[i])
            n_test_labels.append(label2)
            n_test -= 1
        i = i+1
    return n_train_images, n_train_labels, n_test_images, n_test_labels

In [11]:
# getting a dataset of numbers 3 and 5 with train dataset of 80 and test dataset of 35
train_images, train_labels, test_images, test_labels = trainTestData(train_images, train_labels, test_images, test_labels, 3, 5, 80, 35)

In [12]:
# this functions makes the train dataset balanced
# number of images for number 3 same as that for number 5
def balanceDataSet(BAS, train_labels):
    new_train_images_3 = []
    new_train_images_5 = []

    new_train_labels_3 = []
    new_train_labels_5 = []

    for i in range(len(BAS)):
      if train_labels[i] == 3:
        new_train_images_3.append(BAS[i])
        new_train_labels_3.append(3)
      else:
        new_train_images_5.append(BAS[i])
        new_train_labels_5.append(5)
    return new_train_images_3[0:30] + new_train_images_5[0:30], new_train_labels_3[0:30] + new_train_labels_5[0:30]

In [13]:
train_images, train_labels = balanceDataSet(train_images, train_labels)
# to have a fair comparison of performance
org_train_images, org_train_labels, org_test_images, org_test_labels = train_images, train_labels, test_images, test_labels

In [14]:
print("length of train images", len(train_images))
print("length of test images", len(test_images))

length of train images 60
length of test images 35


In [15]:
train_images = np.array(train_images)
test_images = np.array(test_images)
train_labels = np.array(train_labels)
test_labels = np.array(test_labels)
train_images.shape

(60, 28, 28, 1)

In [16]:
model = Sequential()
# 784 is 28*28
# 10 is the number of neurons
model.add(Dense(10, activation='relu', input_shape=(784,)))
model.add(Dense(10, activation='relu'))
# sigmoid activation used for binary classification
model.add(Dense(2, activation='sigmoid'))

In [17]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [18]:
# changing data shape from (28, 28) t0 (784, )
train_data = train_images.reshape((60, 784))

#encoding train labels using one-hot encoding
categories = np.array([3, 5])
encoded_train_labels = np.zeros((len(train_labels), len(categories)))
for i in range(len(train_labels)):
  j = np.where(categories == train_labels[i])
  encoded_train_labels[i, j] = 1

In [19]:
model.reset_states()
model.fit(train_data, encoded_train_labels, validation_split=0.2, epochs=10)

Epoch 1/10
2/2 [==============================] - 1s 326ms/step - loss: 0.6514 - accuracy: 0.6250 - val_loss: 0.8272 - val_accuracy: 0.0833
Epoch 2/10
2/2 [==============================] - 0s 47ms/step - loss: 0.6069 - accuracy: 0.6250 - val_loss: 0.8420 - val_accuracy: 0.0833
Epoch 3/10
2/2 [==============================] - 0s 105ms/step - loss: 0.5725 - accuracy: 0.6458 - val_loss: 0.8716 - val_accuracy: 0.0000e+00
Epoch 4/10
2/2 [==============================] - 0s 66ms/step - loss: 0.5427 - accuracy: 0.6667 - val_loss: 0.8921 - val_accuracy: 0.0000e+00
Epoch 5/10
2/2 [==============================] - 0s 80ms/step - loss: 0.5155 - accuracy: 0.6875 - val_loss: 0.9111 - val_accuracy: 0.0000e+00
Epoch 6/10
2/2 [==============================] - 0s 76ms/step - loss: 0.4902 - accuracy: 0.7292 - val_loss: 0.9202 - val_accuracy: 0.0000e+00
Epoch 7/10
2/2 [==============================] - 0s 65ms/step - loss: 0.4676 - accuracy: 0.7917 - val_loss: 0.9228 - val_accuracy: 0.0000e+00
Epoch

In [20]:
# changing data shape from (28, 28) t0 (784, )
test_data = test_images.reshape((len(test_labels), 784))

# encoding test labels using one-hot encoding
categories = np.array([3, 5])
encoded_test_labels = np.zeros((len(test_labels), len(categories)))
for i in range(len(test_labels)):
  j = np.where(categories == test_labels[i])
  encoded_test_labels[i, j] = 1

In [21]:
model.evaluate(test_data, encoded_test_labels)

2/2 [==============================] - 0s 7ms/step - loss: 0.6756 - accuracy: 0.6000


[0.6755664944648743, 0.6000000238418579]

In [22]:
predicted = model.predict(test_data)
predicted

2/2 [==============================] - 0s 4ms/step


array([[0.27734378, 0.30535358],
       [0.6295132 , 0.35980883],
       [0.6698069 , 0.19054408],
       [0.34620476, 0.33281693],
       [0.648744  , 0.26224458],
       [0.6286365 , 0.2548647 ],
       [0.5376157 , 0.347593  ],
       [0.46365857, 0.3114888 ],
       [0.62262326, 0.16585965],
       [0.2992922 , 0.2809639 ],
       [0.51064026, 0.40471   ],
       [0.3784774 , 0.38932922],
       [0.4949605 , 0.32660776],
       [0.5524255 , 0.2688919 ],
       [0.6179524 , 0.33268133],
       [0.61796993, 0.279222  ],
       [0.67918026, 0.3037739 ],
       [0.6231152 , 0.20163053],
       [0.4195787 , 0.27355427],
       [0.44293156, 0.33457854],
       [0.65076715, 0.22650471],
       [0.23885763, 0.26846334],
       [0.30575523, 0.27949044],
       [0.48719233, 0.17950593],
       [0.6415527 , 0.2808504 ],
       [0.3230934 , 0.34346452],
       [0.48923168, 0.37632033],
       [0.25025585, 0.27062172],
       [0.56161666, 0.32995626],
       [0.32797948, 0.271203  ],
       [0.

<h2>Using 4x4 images (quantum filter)</h2>

In [23]:
train_images, train_labels, test_images, test_labels = org_train_images, org_train_labels, org_test_images, org_test_labels

In [24]:
# Applying the quantum filter for the first time
# (28, 28) to (14, 14) image
start_time = time.time()

filtered_train_images = apply_filter(train_images, 6, data.n_channels, data.n_layers, True)
filtered_test_images = apply_filter(test_images, 6, data.n_channels, data.n_layers, True)

end_time = time.time()

time_taken = end_time - start_time

print(time_taken/60, " minutes")

fitlered pre-processing of images, with type = 6
fitlered pre-processing of images, with type = 6
1.591624395052592  minutes


In [25]:
train_images = filtered_train_images
test_images = filtered_test_images

In [26]:
print("shape of train images", train_images[0].shape)
print("shape of test images", test_images[0].shape)

shape of train images (14, 14, 1)
shape of test images (14, 14, 1)


In [27]:
# Applying the quantum filter for the second time
# (14, 14) to (7,7) image
start_time = time.time()

filtered_train_images = apply_filter(train_images, 6, data.n_channels, data.n_layers, True)
filtered_test_images = apply_filter(test_images, 6, data.n_channels, data.n_layers, True)

end_time = time.time()

time_taken = end_time - start_time

print(time_taken/60, " minutes")

fitlered pre-processing of images, with type = 6
fitlered pre-processing of images, with type = 6
0.392350705464681  minutes


In [28]:
train_images = filtered_train_images
test_images = filtered_test_images

In [29]:
print("shape of train images", train_images[0].shape)
print("shape of test images", test_images[0].shape)

shape of train images (7, 7, 1)
shape of test images (7, 7, 1)


In [30]:
# function to perform max pooling using keras
def conv(train_images, test_images):

    new_train_images, new_test_images = [], []

    for img in train_images:
        img = np.reshape(img, [1, 7, 7, 1])
        max_pool_2d = keras.layers.MaxPooling2D(pool_size=(1, 1), strides=(2, 2), padding="valid")
        new_train_images.append(np.reshape(max_pool_2d(img), [4, 4]))

    for img in test_images:
        img = np.reshape(img, [1, 7, 7, 1])
        max_pool_2d = keras.layers.MaxPooling2D(pool_size=(1, 1), strides=(2, 2), padding="valid")
        new_test_images.append(np.reshape(max_pool_2d(img), [4, 4]))
    return new_train_images, new_test_images

In [31]:
train_images, test_images = conv(train_images, test_images)

In [32]:
train_images = np.array(train_images)
test_images = np.array(test_images)
train_labels = np.array(train_labels)
test_labels = np.array(test_labels)
train_images.shape

(60, 4, 4)

In [33]:
# 16 used instead of 784 since 4*4 = 16
train_data = train_images.reshape((60, 16))

categories = np.array([3, 5])
encoded_train_labels = np.zeros((len(train_labels), len(categories)))
for i in range(len(train_labels)):
  j = np.where(categories == train_labels[i])
  encoded_train_labels[i, j] = 1

In [34]:
model2 = Sequential()
model2.add(Dense(10, activation='relu', input_shape=(16,)))
model2.add(Dense(10, activation='relu'))
model2.add(Dense(2, activation='sigmoid'))

In [35]:
model2.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [36]:
model2.reset_states()
model2.fit(train_data, encoded_train_labels, validation_split=0.2, epochs=10)

Epoch 1/10
2/2 [==============================] - 1s 267ms/step - loss: 0.8494 - accuracy: 0.4167 - val_loss: 0.4454 - val_accuracy: 0.9167
Epoch 2/10
2/2 [==============================] - 0s 72ms/step - loss: 0.8292 - accuracy: 0.4167 - val_loss: 0.4609 - val_accuracy: 0.8333
Epoch 3/10
2/2 [==============================] - 0s 42ms/step - loss: 0.8129 - accuracy: 0.4375 - val_loss: 0.4772 - val_accuracy: 0.8333
Epoch 4/10
2/2 [==============================] - 0s 43ms/step - loss: 0.7976 - accuracy: 0.4583 - val_loss: 0.4942 - val_accuracy: 0.8333
Epoch 5/10
2/2 [==============================] - 0s 44ms/step - loss: 0.7807 - accuracy: 0.4583 - val_loss: 0.5109 - val_accuracy: 0.8333
Epoch 6/10
2/2 [==============================] - 0s 46ms/step - loss: 0.7653 - accuracy: 0.4792 - val_loss: 0.5280 - val_accuracy: 0.8333
Epoch 7/10
2/2 [==============================] - 0s 56ms/step - loss: 0.7521 - accuracy: 0.5208 - val_loss: 0.5460 - val_accuracy: 0.8333
Epoch 8/10
2/2 [==========

In [37]:
# 16 used instead of 784 since 4*4 = 16
test_data = test_images.reshape((len(test_labels), 16))

categories = np.array([3, 5])
encoded_test_labels = np.zeros((len(test_labels), len(categories)))
for i in range(len(test_labels)):
  j = np.where(categories == test_labels[i])
  encoded_test_labels[i, j] = 1

In [38]:
model2.evaluate(test_data, encoded_test_labels)

2/2 [==============================] - 0s 10ms/step - loss: 0.6949 - accuracy: 0.5714


[0.6949045062065125, 0.5714285969734192]